In [1]:
# Dependencies
import pandas as pd
from bs4 import BeautifulSoup
import requests
from splinter import Browser
from webdriver_manager.chrome import ChromeDriverManager
import time

In [8]:
# Setup splinter
executable_path = {'executable_path': ChromeDriverManager().install() }
browser = Browser('chrome', **executable_path, headless=False)

[WDM] - Current google-chrome version is 87.0.4280
[WDM] - Get LATEST driver version for 87.0.4280
[WDM] - Driver [C:\Users\User\.wdm\drivers\chromedriver\win32\87.0.4280.88\chromedriver.exe] found in cache


### Scrape latest headline from Nasa Mars news

In [9]:
# Visit the URL of page to be scraped
nasa_url = 'https://mars.nasa.gov/news/'
browser.visit(nasa_url)

In [10]:
# Parse Results HTML with BeautifulSoup
# Find Everything Inside:
#   <ul class="item_list">
#     <li class="slide">

nasa_html = browser.html
soup_nasa = BeautifulSoup(nasa_html, "html.parser")
required_list = soup_nasa.select_one("ul.item_list li.slide")


In [11]:
required_list.find("div", class_="content_title")

<div class="content_title"><a href="/news/8836/nasa-insights-mole-ends-its-journey-on-mars/" target="_self">NASA InSight's ‘Mole' Ends Its Journey on Mars</a></div>

In [12]:
# Scrape the Latest News Title
title = required_list.find("div", class_="content_title").get_text()
print(title)

NASA InSight's ‘Mole' Ends Its Journey on Mars


In [13]:
# Scrape the Latest Paragraph Text
title_paragraph = required_list.find("div", class_="article_teaser_body").get_text()
print(title_paragraph)

The heat probe hasn’t been able to gain the friction it needs to dig, but the mission has been granted an extension to carry on with its other science.


### Scrape the featured image on JPL, Nasa

In [14]:
# Visit the URL for JPL Featured Space Image 
jpl_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_url)

In [9]:
# Parse Results HTML with BeautifulSoup
jpl_html = browser.html
soup_image = BeautifulSoup(jpl_html, "html.parser")

In [15]:
url_img = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(url_img)

link_bs = BeautifulSoup(browser.html,'lxml')
    
    # Navigate to navbar and click on 4th button 'Galleries'
buttons = browser.find_by_tag('button')
buttons[4].click()
bs2 = BeautifulSoup(browser.html,'lxml')

    # Click on FEATURED IMAGE link
img= bs2.find_all(class_='text-subtitle-sm mb-2')[0].text.replace('\n','').strip()
browser.links.find_by_partial_text(img).click()

    # Click on Download JPG button to obtain full size image url
bs3 = BeautifulSoup(browser.html,'lxml')
browser.links.find_by_partial_text('Download JPG').click()

    # Link for full size url
bs4 = BeautifulSoup(browser.html,'lxml')
featured_url = bs4.find_all('img')[0]['src']

featured_url

'https://d2pn8kiwq2w21t.cloudfront.net/original_images/jpegPIA24179.jpg'

### Scrape Mars space facts table

In [20]:
# Visit the Mars Facts webpage 
mars_url = 'https://space-facts.com/mars/'
browser.visit(mars_url)

In [21]:
# Parse Results HTML with BeautifulSoup
mars_html = browser.html
soup_mars = BeautifulSoup(mars_html, "html.parser")

In [22]:
tables = pd.read_html(mars_url)
len(tables)

3

In [23]:
facts_table = tables[0]
facts_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [28]:
facts_table.columns=['Description', 'Value']
facts_table

,Description,Value
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


### Scrape images and titles of 4 hemispheres of Mars

In [24]:
# Visit the USGS Astrogeology site
astro_url = 'https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars)'
browser.visit(astro_url)

In [25]:
# HTML Object
html_hemispheres = browser.html

# Parse HTML with Beautiful Soup
soup = BeautifulSoup(html_hemispheres, 'html.parser')

# Retreive all items that contain mars hemispheres information
items = soup.find_all('div', class_='item')

# Create empty list for hemisphere urls 
hemisphere_image_urls = []

# Store the main_ul 
hemispheres_main_url = 'https://astrogeology.usgs.gov'

# Loop through the items previously stored
for i in items: 
    # Store title
    title = i.find('h3').text
    
    # Store link that leads to full image website
    partial_img_url = i.find('a', class_='itemLink product-item')['href']
    
    # Visit the link that contains the full image website 
    browser.visit(hemispheres_main_url + partial_img_url)
    
    # HTML Object of individual hemisphere information website 
    partial_img_html = browser.html
    
    # Parse HTML with Beautiful Soup for every individual hemisphere information website 
    soup = BeautifulSoup( partial_img_html, 'html.parser')
    
    # Retrieve full image source 
    img_url = hemispheres_main_url + soup.find('img', class_='wide-image')['src']
    
    # Append the retreived information into a list of dictionaries 
    hemisphere_image_urls.append({"title" : title, "img_url" : img_url})
    

# Display hemisphere_image_urls
hemisphere_image_urls

[{'title': 'Cerberus Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/f5e372a36edfa389625da6d0cc25d905_cerberus_enhanced.tif_full.jpg'},
 {'title': 'Schiaparelli Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/3778f7b43bbbc89d6e3cfabb3613ba93_schiaparelli_enhanced.tif_full.jpg'},
 {'title': 'Syrtis Major Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/555e6403a6ddd7ba16ddb0e471cadcf7_syrtis_major_enhanced.tif_full.jpg'},
 {'title': 'Valles Marineris Hemisphere Enhanced',
  'img_url': 'https://astrogeology.usgs.gov/cache/images/b3c7c6c9138f57b4756be9b9c43e3a48_valles_marineris_enhanced.tif_full.jpg'}]

In [5]:
browser.quit()